In [2]:
# New Stuff
from torchtext.data import Field, TabularDataset, BucketIterator, LabelField
import spacy
import torch
import random
import numpy as np


# Don't Really Need This Line, Ensures Reproducibility
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# We are loading our text data into a text field, ignore title for now. batch_first makes tensors with the batch
# dimension first. You don't have to set lower = True (it is default false), You could set use_vocab = True (but true
# default anyways). Same with sequential. See here: https://pytorch.org/text/_modules/torchtext/data/field.html.
# Tokenization is simply how we choose to tokenize the text data or split it into distinct smaller units.
TEXT = Field(batch_first = True, tokenize='spacy', lower=True)

#Basically here the dtype = torch.float rather than torch.long which is the default because 
#torch.float allows us to create the loss function later which only accepts float tensors.
LABEL = LabelField(dtype = torch.float, tokenize='spacy', lower=True)

#We are just definining the batch attribute name here: TEXT and LABEL when reading in our JSON data. All that happens
#is that TEXT becomes the batch attribute that gives us text data. When so when we define our iterator we can acesss
#the text data using batch.TEXT. Same thing when we build our vocab. Note the below is a python dictionary with the
#tuple representing the attribute we will use for our bathces. The key should match the JSON data.
fields = {'text': ('text', TEXT), 'label': ('label', LABEL)}

/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [3]:
#Our train_data is basically just a big array of dictionaries. Each dictionary has keys, in our case
#just text : "text goes here" and for test_data its just label: "fake/real". So "label" is our key, and "text" is a
#a key. However train_data (which is some sort of Dataset object) has __dict__ as an attribute. I'm not exactly sure what the type of train_data is.
#I'm guessing its just a TabularDataset. Actually TabularDataset extends DataSet.
#Oh a TabularDataset is just a normal DataSet but the columns are stored in CSV, TSV, or JSON format.
#This has all the documentation for TabularDataset and BucketIterator that I can find: https://torchtext.readthedocs.io/en/latest/data.html.
#There is also this: the actual github docs: https://github.com/pytorch/text/blob/master/torchtext/data/dataset.py
train_data, test_data = TabularDataset.splits(
    path='../data',
    train='news.csv',
    test='news.csv',
    format='csv',
    fields=fields)
print("Num of training: ", len(train_data))
print("Num of testing: ", len(test_data))

/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


Num of training:  6335
Num of testing:  6335


In [4]:
type(train_data)

torchtext.data.dataset.TabularDataset

In [3]:

train_data, valid_data = train_data.split(split_ratio=0.75)
print("Num of training: ", len(train_data))
print("Num of validation: ", len(valid_data))
print("Num of testing: ", len(test_data))

Num of training:  4751
Num of validation:  1584
Num of testing:  6335


In [4]:
print(train_data[0].__dict__.keys())
print(train_data[0].__dict__.values())

dict_keys(['text', 'label'])
dict_values([['on', 'this', 'day', 'in', '1973', ',', 'j.', 'fred', 'buzhardt', ',', 'a', 'lawyer', 'defending', 'president', 'richard', 'nixon', 'in', 'the', 'watergate', 'case', ',', 'revealed', 'that', 'a', 'key', 'white', 'house', 'tape', 'had', 'an', '18', '...'], 'real'])


In [5]:
#Builds vocab for LABELS/TEXT data
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [6]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(TEXT.vocab.freqs.most_common(20))
print(TEXT.vocab.itos[:10])
print(LABEL.vocab.stoi)

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
[('the', 220806), (',', 193711), ('.', 163403), ('to', 106045), ('of', 98354), ('and', 90919), ('a', 81926), ('in', 75274), ('that', 55067), ('\n\n', 43799), ('is', 41983), ('for', 35367), ('-', 32910), ('on', 31077), ('"', 30197), ('it', 30057), ('\n', 29115), ('’s', 26896), ('“', 26056), ('”', 25522)]
['<unk>', '<pad>', 'the', ',', '.', 'to', 'of', 'and', 'a', 'in']
defaultdict(None, {'fake': 0, 'real': 1})


In [7]:
BATCH_SIZE = 64

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort = False)


/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [8]:
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [9]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [10]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2111,  0.0242,  0.3948,  ..., -0.9228,  0.3897,  0.0144],
        [ 0.4726, -0.1936,  0.5946,  ..., -0.2373,  0.8675,  0.4416],
        [-0.3018, -0.3586,  0.0533,  ..., -0.2412, -0.6657, -0.0248]])

In [11]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [12]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [13]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [14]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.TEXT).squeeze(1)
        
        loss = criterion(predictions, batch.LABEL)
        
        acc = binary_accuracy(predictions, batch.LABEL)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [15]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.TEXT).squeeze(1)
            
            loss = criterion(predictions, batch.LABEL)
            
            acc = binary_accuracy(predictions, batch.LABEL)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [17]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01 | Epoch Time: 15m 21s
	Train Loss: 0.133 | Train Acc: 95.41%
	 Val. Loss: 0.050 |  Val. Acc: 97.83%
Epoch: 02 | Epoch Time: 18m 58s
	Train Loss: 0.034 | Train Acc: 99.10%
	 Val. Loss: 0.031 |  Val. Acc: 99.00%
Epoch: 03 | Epoch Time: 19m 30s
	Train Loss: 0.020 | Train Acc: 99.60%
	 Val. Loss: 0.023 |  Val. Acc: 99.12%
Epoch: 04 | Epoch Time: 16m 41s
	Train Loss: 0.015 | Train Acc: 99.62%
	 Val. Loss: 0.020 |  Val. Acc: 99.12%
Epoch: 05 | Epoch Time: 17m 4s
	Train Loss: 0.009 | Train Acc: 99.79%
	 Val. Loss: 0.015 |  Val. Acc: 99.38%


In [18]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.007 | Test Acc: 99.83%
